## Montecarlo for errors

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import pandas as pd
import itertools
import csv
import os
import glob
import math
import random
import scipy
from operator import add
from scipy import stats
from numpy import pi,sqrt,exp

In [142]:
# genero tutte le combo con itertools

n=5 
m=10
N = math.comb(m,n)   #coefficiente binomiale di m ed n
print(N)

combos_indexes = range(1,m+1)
combos = list(itertools.combinations(combos_indexes,n)) 

# converto ciascuna combo in una lista di 1 e 0 e poi nel corrispondente intero

integer_combos = []

for combo in combos:          # ho fatto un casino (verificare)
    temp1= [0] * m
    
    for j in combo:        
        temp1[j-1]=1
        
    temp2_combo=(temp1)  
    temp3_combo= (bin(int(''.join(map(str, temp2_combo)), 2)))
    integer_combos.append(int(temp3_combo,2)) 

print(integer_combos)

252
[992, 976, 968, 964, 962, 961, 944, 936, 932, 930, 929, 920, 916, 914, 913, 908, 906, 905, 902, 901, 899, 880, 872, 868, 866, 865, 856, 852, 850, 849, 844, 842, 841, 838, 837, 835, 824, 820, 818, 817, 812, 810, 809, 806, 805, 803, 796, 794, 793, 790, 789, 787, 782, 781, 779, 775, 752, 744, 740, 738, 737, 728, 724, 722, 721, 716, 714, 713, 710, 709, 707, 696, 692, 690, 689, 684, 682, 681, 678, 677, 675, 668, 666, 665, 662, 661, 659, 654, 653, 651, 647, 632, 628, 626, 625, 620, 618, 617, 614, 613, 611, 604, 602, 601, 598, 597, 595, 590, 589, 587, 583, 572, 570, 569, 566, 565, 563, 558, 557, 555, 551, 542, 541, 539, 535, 527, 496, 488, 484, 482, 481, 472, 468, 466, 465, 460, 458, 457, 454, 453, 451, 440, 436, 434, 433, 428, 426, 425, 422, 421, 419, 412, 410, 409, 406, 405, 403, 398, 397, 395, 391, 376, 372, 370, 369, 364, 362, 361, 358, 357, 355, 348, 346, 345, 342, 341, 339, 334, 333, 331, 327, 316, 314, 313, 310, 309, 307, 302, 301, 299, 295, 286, 285, 283, 279, 271, 248, 244, 242, 

In [143]:
#list all csv files in the directory

csv_files = glob.glob('*.{}'.format('csv'))

nfolds_final= [0]*N                       # inizializzo la lista finale delle coincidenze n-fold 



#loop over the csv files

for file in csv_files:
    

    with open(file, newline='') as f:
#         next(f)
        r = csv.reader(f)
        data = [line for line in r]

    new_data = []

    for row in data:                       # converto le stringhe del file in liste di interi
        innerlist = []
        for j in row:
            innerlist.append(int(j))
        new_data.append(innerlist)

    

    temp=[]                                #    Tolgo le coincidenze (prima colonna) e
                                           #   rimango con una lista di 1 e 0 che converto in 
                                           #   un numero binario.
    for new_row in new_data:
        tempin=[]        
        tempin.append(new_row[0])
        new_row.pop(0)
        tempin.append(int(bin(int(''.join(map(str,new_row)), 2)),2))
        temp.append(tempin)

    

    data_dict={}                            #    Tolgo le coincidenze (prima colonna) e
                                            #   rimango con una lista di 1 e 0 che converto in 
                                            #   un numero binario.
    for item in temp:
        key, value = item[1], item[0]
        data_dict[key] = value
       
    
  
    
    nfolds=[]                               #    Scrivo la lista delle coincidenze riordinando le chiavi 
                                            #    in accordo con integer_combos e prendendo i valori
                                            #     corrispondenti
 
    for integer_combo in integer_combos:
        if integer_combo in list(data_dict.keys()):
            nfolds.append(data_dict[integer_combo])
        else:                                            # Quando una chiave/combo non c'è nel file, il 
                                                         # corrispondente valore viene posto a zero.
            nfolds.append(0)
            
            
        
# ad ogni loop aggiungo la lista di coincidenze provenienti dal file in esame  

    nfolds_final = list( map(add, nfolds_final, nfolds) )

nfolds_final_array = np.array(nfolds_final)

nfolds_errors = np.sqrt(nfolds_final_array)



    
    
#totale delle coincidenze rivelate


total_nfolds= sum(nfolds_final)
total_nfolds_error = np.sqrt(total_nfolds)

print ("Total n-folds = ", total_nfolds)


# frequenze relative

frequencies = nfolds_final_array/total_nfolds
ratios = nfolds_errors/nfolds_final_array

where_are_NaNs = np.isnan(ratios)
ratios[where_are_NaNs] = 0

frequencies_errors = frequencies*( (ratios) + (total_nfolds_error/total_nfolds) )

print(len(frequencies_errors))

Total n-folds =  9814
252


<ipython-input-143-ef36b2c541c9>:88: RuntimeWarning: invalid value encountered in true_divide
  ratios = nfolds_errors/nfolds_final_array


In [144]:
# funzione che calcola la fidelity tra due distribuzioni

def fidelity(p_teo, p_exp):
    sqrt_prod = []
    for i in range(N):
        prd_i = ((p_teo[i])*(p_exp[i]))
        sqrt_prod.append(math.sqrt(prd_i))
    F= sum(sqrt_prod)
    return F

In [145]:
# importo le simulazioni (solo caso indist.)

file = open(r"C:\\Users\\PC\Desktop\\Ph.D\\Loop experiment\\Simulations\\5-in-10-0.15-loss-lori-pattern.csv", 'r')
file_string = np.array([row.strip().split('\t') for row in file][0:], dtype=float)
p = np.array(file_string[:, 0])
G_indist=np.sum(p)
p_indist= p/G_indist
print(G_indist)

0.43863181167554566


In [146]:
print(fidelity(frequencies,p_indist))

0.9387091231019019


In [147]:
# simulazione vera e propria

num_simulations = 10000

fidelities = []

for i in range(num_simulations):

    random_values = []
    
    for j in range(len(frequencies)):
        temp = random.gauss(frequencies[j], frequencies_errors[j])
        if temp<0:
            temp=0
        random_values.append(temp)
        random_values_array = np.array(random_values) 

    fidelities.append(fidelity(p_indist, random_values_array))

In [148]:
# calcolo varianza ed errore statistico

var = np.var(fidelities, ddof=1)
stat_err = np.sqrt(var)/np.sqrt(num_simulations)
print(np.sqrt(var))

0.005836279907747944
